In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as col
import pandas as pd
import math

Define how strongly speed and efficiency are valued

In [ ]:
factor_speed = 0.2
factor_effiecency = 0.8

In [ ]:
G = nx.DiGraph()
config = pd.read_csv("track_config.csv", sep=';')
G.add_node(0,v=0)
coordinate_set_dict = {0: (0,0)}

Config everything needed for the calcuation below:

In [ ]:
section_length = 100

g = 9.81                    #Earth gravity constant

car_mass = 150              #weight of car and driver
diameter_tire = 0.4         #in m
coeff_roll_res = 0.02       #maybe a bit lower
gear_ratio = 5              #How many rotations the motor has to do for one roation of the wheel


#Motor model for series wound dc motor
torque_constant = 132.6       #mNm/A
velocity_constant = 72      #1/(min * V)
anchor_resistance = 0.1      #Ohm
torque_loss = 0.5     #Nm

In [ ]:
for x in range (1, len(config.index)):
    for v in range(config.loc[x, "vmin"], config.loc[x, "vmax"]+1, 5):
        node_name = config.loc[x, "s"]*section_length+v
        G.add_node((node_name), v=v)
        coordinate_set_dict[node_name] = (x, v)
        for vold in range(v-30,v+31,5):
            if G.has_node(config.loc[x-1, "s"]*section_length+vold):
                vold_mps = vold/3.6
                v_mps = v/3.6

                time = 2 * section_length / (v_mps + vold_mps)

                gradient_angle = math.tanh( (config.loc[x, "m above sea"]-config.loc[x-1, "m above sea"] ) / section_length)   #in rad
                f_roll_res = car_mass * g * math.cos(gradient_angle) * coeff_roll_res
                f_slope = car_mass * g * math.sin(gradient_angle)
                f_accel = ((v_mps-vold_mps) / time) * car_mass

                f_total = f_accel + f_slope + f_roll_res
                E_total = f_total * section_length

                motor_rotation_speed = ((v_mps+vold_mps)/2) * gear_ratio * 30 / ((diameter_tire/2) * math.pi)     #rotation speed in 1/min
                motor_torque = (f_total * (diameter_tire/2) / gear_ratio)

                motor_current = (motor_torque+torque_loss)*1000/torque_constant
                motor_voltage = (motor_rotation_speed / velocity_constant) + (anchor_resistance * motor_current)
                E_Motor = (motor_voltage*motor_current*time)

                #Prescale the factors, in order to see the wished results when factor_speed and factor_effiecency are within 0 and 1
                if(E_total > 0):
                    cost_econ = E_Motor / 8000
                else:
                    cost_econ = 0
                cost_speed = time / 36
                G.add_edge((config.loc[x-1, "s"]*section_length+vold), (config.loc[x, "s"]*section_length+v), weight=factor_speed*cost_speed+factor_effiecency*cost_econ)


In [ ]:
nodelist = nx.dijkstra_path(G, 0,config.loc[len(config.index)-1]["s"]*section_length+config.loc[len(config.index)-1]["vmin"])

outputlist = []
for i in range(len(config.index)):
     outputlist.append(G.nodes[nodelist[i]]["v"])

print(outputlist)

output = pd.DataFrame(outputlist)
output.rename(columns={0:"v optimal"}, inplace = True)
print(output)
output.to_csv("OptimalesProfilOutput.csv",sep=';')

Generate adjacency matrix and graphical picture of the Graph

In [ ]:
matrix = nx.to_pandas_adjacency(G)

matrix.to_csv("AdjMatrix.csv", sep=';')


#Graph darstellen
#nx.draw_networkx(G)


weight_list = []
attributelist = nx.get_edge_attributes(G,'weight')
for edge in G.edges:
    weight_list.append(attributelist[edge])
cmap = col.LinearSegmentedColormap.from_list("grey shades",['red','black','grey','lightgrey'])

nx.draw(G, pos=coordinate_set_dict, with_labels=False, node_color='blue', edge_color=weight_list, edge_cmap=cmap)

plt.show()
